# Dreval demo
You can execute the DrEval Framework either via Nextflow as nf-core pipeline or as Python standalone.

In [ ]:
!pip install drevalpy

## Nextflow demo

You should use DrEval with Nextflow on high-performance clusters or clouds. Nextflow supports various systems like Slurm, AWS, Azure, Kubernetes, or SGE. On a local machine, you can also use the pipeline but probably, the overhang from spawning processes is not worth it so you might prefer the standalone. Nextflow needs a java version >=17, so we need to install that, too.

In [31]:
!pip install nextflow
!apt-get install openjdk-17-jre-headless -qq > /dev/null
!java --version

openjdk 17.0.15 2025-04-15
OpenJDK Runtime Environment (build 17.0.15+6-Ubuntu-0ubuntu122.04)
OpenJDK 64-Bit Server VM (build 17.0.15+6-Ubuntu-0ubuntu122.04, mixed mode, sharing)


In [ ]:
# we need a demo config for nextflow because on colab, we only have two CPUs available:
with open('demo.config', 'w') as f:
  f.write('process {\n')
  f.write('\tresourceLimits = [\n')
  f.write('\t\tcpus: 2,\n')
  f.write('\t\tmemory: "3.GB",\n')
  f.write('\t\ttime: "1.h",\n')
  f.write('\t]\n')
  f.write('}')

We run the pipeline with the TOYv1 dataset which was subset from CTRPv2. For the demo, we don't do hyperparameter tuning and we just do 2 CV splits. We want to inspect the final model which is why we train a final model on the full dataset.


In [37]:
!nextflow run nf-core/drugresponseeval -r dev -c demo.config --dataset_name TOYv1 --models ElasticNet --baselines NaiveMeanEffectsPredictor --n_cv_splits 2 --no_hyperparameter_tuning --final_model_on_full_data


 N E X T F L O W   ~  version 25.04.4

Launching `https://github.com/nf-core/drugresponseeval` [fabulous_poitras] DSL2 - revision: 00ebfddaa6 [dev]


------------------------------------------------------
                                        ,--./,-.
        ___     __   __   __   ___     /,-._.--~'
  |\ | |__  __ /  ` /  \ |__) |__         }  {
  | \| |       \__, \__/ |  \ |___     \`-._,-`-,
                                        `._,._,'
  nf-core/drugresponseeval 1.1.0
------------------------------------------------------
Model options
  models                  : ElasticNet

Input/output options
  dataset_name            : TOYv1

Additional options
  n_cv_splits             : 2
  no_hyperparameter_tuning: true
  final_model_on_full_data: true

Generic options
  trace_report_suffix     : 2025-06-25_12-10-13

Core Nextflow options
  revision                : dev
  runName                 : fabulous_poitras
  container               : ghcr.io/daisybio/drevalpy:v1.3.4
  launchDi

The results will be stored in `results/my_run`. You can inspect pipeline information like runtime or memory in `results/pipeline_info`. In `my_run/report`, you can find the html report where you can look at your results interactively. The underlying data is in `my_run/evaluation_results.csv` or `true_vs_pred.csv`.

We now inspect the final model saved in `results/my_run/LCO/ElasticNet/final_model` with `drevalpy` functions.

In [83]:
from drevalpy.models import MODEL_FACTORY
enet_class = MODEL_FACTORY["ElasticNet"]
enet = enet_class.load("results/my_run/LCO/ElasticNet/final_model")
enet

We now want to extract the top scoring features.

In [84]:
# get the top features
cell_line_input = enet.load_cell_line_features(data_path="data", dataset_name="TOYv1")
drug_input = enet.load_drug_features(data_path="data", dataset_name="TOYv1")
all_features = list(cell_line_input.meta_info['gene_expression'])+[f'fingerprint_{i}' for i in range(128)]

In [87]:
import pandas as pd
df = pd.DataFrame({'feature': all_features, 'coef': enet.model.coef_})
df.sort_values(by="coef", ascending=False)

,feature,coef
900,fingerprint_33,0.921525
942,fingerprint_75,0.618429
983,fingerprint_116,0.602301
911,fingerprint_44,0.510778
890,fingerprint_23,0.489396
...,...,...
932,fingerprint_65,-0.561930
939,fingerprint_72,-0.608778
990,fingerprint_123,-0.675462
895,fingerprint_28,-0.695587


In [93]:
print("Top 50 features:")
list(df.sort_values(by="coef", ascending=False)["feature"][:50])

Top 50 features:


['fingerprint_33',
 'fingerprint_75',
 'fingerprint_116',
 'fingerprint_44',
 'fingerprint_23',
 'fingerprint_120',
 'fingerprint_4',
 'fingerprint_69',
 'fingerprint_57',
 'fingerprint_31',
 'fingerprint_109',
 'fingerprint_61',
 'fingerprint_112',
 'fingerprint_50',
 'fingerprint_126',
 'fingerprint_63',
 'fingerprint_122',
 'fingerprint_47',
 'fingerprint_14',
 'fingerprint_43',
 'fingerprint_85',
 'fingerprint_20',
 'fingerprint_55',
 'fingerprint_107',
 'fingerprint_121',
 'fingerprint_37',
 'fingerprint_24',
 'fingerprint_110',
 'fingerprint_30',
 'fingerprint_91',
 'fingerprint_62',
 'fingerprint_53',
 'fingerprint_78',
 'fingerprint_64',
 'fingerprint_38',
 'fingerprint_92',
 'fingerprint_82',
 'fingerprint_52',
 'fingerprint_66',
 'fingerprint_3',
 'fingerprint_115',
 'fingerprint_59',
 'fingerprint_18',
 'fingerprint_118',
 'fingerprint_83',
 'fingerprint_87',
 'fingerprint_8',
 np.str_('PLK1'),
 np.str_('PIK3CA'),
 'fingerprint_56']

The fingerprints are the most important features as the drug identity is responsible for the most variation between responses.

## Standalone demo

In [ ]:
from drevalpy.models import MODEL_FACTORY
from drevalpy.datasets import AVAILABLE_DATASETS
# First let us see which dataset and models are already implemented in drevalpy. You can test your own model on all the datasets and comapre your model to all.the implemented ones:
print(f"Models: {list(MODEL_FACTORY.keys())}")
print(f"Dataset: {list(AVAILABLE_DATASETS.keys())}")

# let us first train amodel on the toy dataset. It will download the dataset for you.
from drevalpy.experiment import drug_response_experiment

naive_mean = MODEL_FACTORY["NaivePredictor"]
rf = MODEL_FACTORY["ElasticNet"]
simple_nn = MODEL_FACTORY["SimpleNeuralNetwork"]

toyv2 = AVAILABLE_DATASETS["TOYv2"](path_data="data", measure="LN_IC50_curvecurator")

drug_response_experiment(
            models=[rf, simple_nn],
            baselines=[naive_mean],
            response_data=toyv2,
            metric="RMSE",
            n_cv_splits=2,
            test_mode="LCO",
            run_id="my_first_run",
            path_data="data",
            hyperparameter_tuning=False,
        )

TOYv2 data downloaded and extracted to data
Creating cv splits at results/my_second_run/TOYv2/LCO/splits
Running ElasticNet
- Full Test -
################# FOLD 1/7 #################
Best hyperparameters: {'alpha': 1, 'l1_ratio': 0}
Training model on full train and validation set to predict test set
Loading cell line features ...
Loading drug features ...
Number of cell lines in features: 90
Number of drugs in features: 36
Number of cell lines in train dataset: 78
Number of drugs in train dataset: 36
Reduced training dataset from 1050 to 1009, due to missing features
Reduced prediction dataset from 171 to 171, due to missing features
Training model ...
Using temporary directory: /tmp/tmpahtzfvvv for model checkpoints
################# FOLD 2/7 #################
Best hyperparameters: {'alpha': 1, 'l1_ratio': 0}
Training model on full train and validation set to predict test set
Loading cell line features ...
Loading drug features ...
Number of cell lines in features: 90
Number of drugs 

/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


Loading drug features ...
Number of cell lines in features: 90
Number of drugs in features: 36
Number of cell lines in train dataset: 76
Number of drugs in train dataset: 36
Reduced early stopping dataset from 21 to 21
Training model ...
Using temporary directory: /tmp/tmp7s80hyg7 for model checkpoints


INFO:pytorch_lightning.callbacks.model_summary:
  | Name                   | Type       | Params | Mode 
--------------------------------------------------------------
0 | loss                   | MSELoss    | 0      | train
1 | fully_connected_layers | ModuleList | 32.6 K | train
2 | batch_norm_layers      | ModuleList | 120    | train
3 | dropout_layer          | Dropout    | 0      | train
--------------------------------------------------------------
32.7 K    Trainable params
0         Non-trainable params
32.7 K    Total params
0.131     Total estimated model params size (MB)
13        Modules in train mode
0         Modules in eval mode


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

################# FOLD 4/7 #################
Best hyperparameters: {'dropout_prob': 0.3, 'max_epochs': 100, 'units_per_layer': [32, 16, 8, 4], 'input_dim_gex': 867, 'input_dim_fp': 128}
Training model on full train and validation set to predict test set
Loading cell line features ...
Loading drug features ...
Number of cell lines in features: 90
Number of drugs in features: 36
Number of cell lines in train dataset: 75
Number of drugs in train dataset: 36
Reduced early stopping dataset from 22 to 22
Training model ...
Using temporary directory: /tmp/tmppj4tf8kf for model checkpoints


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 8 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.callbacks.model_summary:
  | Name                   | Type       | Params | Mode 
--------------------------------------------------------------
0 | loss                   | MSELoss    | 0      | train
1 | fully_connected_layers | ModuleList | 32.6 K | train
2 | batch_norm_layers  

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:
Detected KeyboardInterrupt, attempting graceful shutdown ...


In [ ]:
#run drevalpy in the console: